In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import (confusion_matrix, classification_report, plot_roc_curve, roc_auc_score, 
accuracy_score, precision_score, recall_score, f1_score, auc, precision_recall_curve, average_precision_score)

from imblearn.under_sampling import NearMiss


In [25]:
#Load datasets
train = pd.read_csv('../CapstoneTwo/train_final.csv')
test = pd.read_csv('../CapstoneTwo/test_final.csv')


In [26]:
#Get X and y
X = train.drop(columns='WnvPresent')._get_numeric_data()
y = train['WnvPresent']

#### Baseline Prediction

Our data is imbalanced. If we were to predict "West Nile Virus not Present," we would be roughly 95% correct. Though accuracy is high, this is not a good approach.

#### Model 1: Logistic Regression

In [27]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [28]:
lr_clf = LogisticRegression(random_state=42)

In [29]:
#Create our pipeline
from sklearn.pipeline import make_pipeline
lr_pipe = make_pipeline(scaler, lr_clf)

In [30]:
scores = cross_val_score(lr_pipe, X, y, scoring='roc_auc', cv=5)

/Users/louistomczak/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/louistomczak/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

In [31]:
print(scores.mean())

0.5595626387539999


Let's find our optimal tuning parameters

In [32]:
#params = {}
#params['LogisticRegression_penalty'] = ['l1', 'l2']

In [33]:
#grid = GridSearchCV(lr_pipe, params, cv=5, scoring='roc_auc')
#grid.fit(X,y)

In [34]:
#grid.best_score_

In [35]:
#run and check score/confusion matrix etc.

In [36]:
#bootstrap?

#### 2. Random Forest

In [37]:
rf_clf = RandomForestClassifier(random_state=42)

In [38]:
rf_pipe = make_pipeline(scaler, rf_clf)

In [39]:
scores = cross_val_score(rf_pipe, X, y, scoring='roc_auc', cv=5)

In [40]:
print(scores.mean())

0.3494986859376613


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [42]:
#nm = NearMiss(version=3)
#X_train, y_train = nm.fit_resample(X_train, y_train)

In [43]:
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [44]:
y_pred = rf_pipe.predict(X_test)

In [23]:
roc_auc_score(y_test, y_pred)

0.6336254107338445

In [23]:
#Hyperparameters

In [24]:
#Gridsearch Cv

In [25]:
#Get best estimator and fit

In [26]:
#Interpret

#### 3. XG Boost Classifier

In [27]:
scaler = StandardScaler()
xgb_clf = xgb.XGBClassifier()

In [28]:
xgb_pipe = make_pipeline(scaler, xgb_clf)

In [29]:
scores = cross_val_score(xgb_pipe, X, y, scoring='roc_auc', cv=5)

In [30]:
print(scores)

[0.57249741 0.57343729 0.52267933 0.20090407 0.65307749]


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [32]:
nm = NearMiss(version=3)
X_train, y_train = nm.fit_resample(X_train, y_train)

In [33]:
xgb_pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [34]:
y_pred = xgb_pipe.predict(X_test)

In [35]:
roc_auc_score(y_test, y_pred)

0.6432867834976268

In [ ]:
#Explain XG Boost with Shapley

#### Pipeline

xgb_pipe = Pipeline([('st_scaler', StandardScaler()),
('xgb_model'), xgb.XGBClassifier()])

scores = cross_val_score(xgb_pipeline, X, y, scoring = AUC, cv=5)

#### Tuning the XGB pipe

param grid